In [14]:
%pylab inline
import numpy as np
import scipy as sc

#import seaborn as sns
#sns.set_style("whitegrid")
#sns.set_context("talk")
#rc('axes', labelsize=20, titlesize=20)

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math
import scipy.stats as ss

Populating the interactive namespace from numpy and matplotlib


In [15]:
def SimulateNormal(mean, sd, N):
    return np.random.normal(loc=mean, scale=sd, size=N)

def PlotPosteriorAgainstSample(priorMean, priorSd, likelihoodSd, sampledTheta, originalData):
    Hist = plt.hist(sampledTheta, sd(len(sampledTheta)), normed=True) #plot the hist for the sampled theta 
    mean = (priorMean/(priorSd^2) + np.sum(originalData)/likelihoodSd^2)/(1/priorSd^2 + len(originalData)/likelihoodSd^2)
    sd = (1/priorSd^2 + len(originalData)/likelihoodSd^2)^(-1)
    x = np.linspace(mean-3*sd, mean+3*sd, 100)
    trueDensity = plt.plot(x,mlab.normpdf(x, mean, sd)) #plot the true density with mean and sd               
    plt.show()


In [38]:
def ABC(priorSampler, likelihoodSimulator, summaryStatistics, epsilon, data, n): # epsilon is the number
    prior_sample = priorSampler(n)
    stat = summaryStatistics(data)
    theta_generated = []
    accepted = []
    output_list = []
    
    for i in range(n):
        # we assume data is a list of vectors and those vectors are columns of our 'data frame' with data
        simulated_data = likelihoodSimulator(len(data), prior_sample[i])
        temporary_stat = summaryStatistics(simulated_data)
        if abs(temporary_stat - stat)<epsilon:
            accept = 1
        else: accept = 0
            
        output_dict = {'accept': accept, 'z':simulated_data}
        output_list.append(output_dict)
    
    return pd.DataFrame(output_list)

In [40]:
def NormalPriorSampler(n):
    return list(SimulateNormal(0,1,n))

def NormalLiklihoodSimulator(n, param):
    #unknown mean
    return list(SimulateNormal(param, 1,n))
    
def NormalSummary(data):
    return float(np.mean(data))

data = list(np.random.normal(0,1,10))
print NormalSummary(data)
df = ABC(NormalPriorSampler, NormalLiklihoodSimulator, NormalSummary, 0.1, data , 100)
print df

0.437234283507
    accept                                                  z
0        0  [0.689549319739, -1.53016468326, -0.1431884011...
1        0  [1.26337267623, -0.233177595014, 1.2896884123,...
2        0  [1.28357770067, -0.0612303573895, 0.0022987629...
3        0  [-1.02434389828, 1.2767716525, 1.33361880263, ...
4        0  [0.476155506531, 3.15955970207, 0.29358999006,...
5        1  [0.0798716808763, -0.806878843242, 0.633573275...
6        0  [-1.5319805636, -0.0784463529289, 0.0493384310...
7        1  [0.698171629866, 0.017626417018, -0.2916556793...
8        0  [1.11265426469, 2.55571436105, 2.22154593057, ...
9        0  [0.287981265759, -0.183783243256, 0.3276106225...
10       0  [1.08420741661, 0.913656588444, 2.36434992017,...
11       0  [2.57180301805, 0.849974486652, 0.052468624348...
12       0  [-2.58859879791, 0.488804210874, 0.04698977961...
13       0  [0.369700397748, -1.28793983079, -0.3028861250...
14       0  [-0.0700210640324, -1.99451436932, 0.086464